# Table of Contents

- **[Load Library](#Load-Library)**
- **[Load Data](#Load-Data)**
- **[EDA](#EDA)**
- **[Feature Extraction](#Feature-Extraction)**
- **[Modeling](#Modeling)**
- **[Submission](#Submission)**

# Load Library

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_colwidth", 80)
import os
import matplotlib.pyplot as plt
import seaborn as sns
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from wordcloud import STOPWORDS
from collections import defaultdict
import string
from sklearn.feature_extraction.text import TfidfVectorizer

# Load Data

In [ ]:
train = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")
print('Train Set Shape = {}'.format(train.shape))
train.head(3)

In [ ]:
test = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/test.csv")
print('Test Set Shape = {}'.format(test.shape))
test.head(3)

In [ ]:
submission = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/sample_submission.csv")
submission.head(3)

# EDA

## Target Distribution

In [ ]:
print(f'the number of insincere questions is : {len(train[train["target"]==1])} / {len(train)}')
print(f'the number of not-insincere questions is : {len(train[train["target"]==0])} / {len(train)}')
train_counts = train["target"].value_counts()
hv.Bars((train_counts.keys(), train_counts.values),"Target Label","Counts").opts(width=600,height=400,title="Target Counts",tools=['hover'])

## Null Ratio

In [ ]:
print(f'the number of nulls in train set : {train.isnull().any().sum()}')
print(f'the number of nulls in test set : {test.isnull().any().sum()}')

## N-gram Frequencies

In [ ]:
#ngram function
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

In [ ]:
train_0 = train[train["target"]==0]
train_1 = train[train["target"]==1]

### Unigram

In [ ]:
freq_dict_0_uni = defaultdict(int)
for sent in train_0["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict_0_uni[word] += 1

freq_dict_1_uni = defaultdict(int)
for sent in train_1["question_text"]:
    for word in generate_ngrams(sent):
        freq_dict_1_uni[word] += 1

In [ ]:
data_0_uni = list(sorted(freq_dict_0_uni.items(), key=lambda x: x[1],reverse=True))
bars_0_uni = hv.Bars(data_0_uni[0:50][::-1],"Word","Count").opts(invert_axes=True, width=500, height=800, title="unigram frequency with target=0", color="red")
data_1_uni = list(sorted(freq_dict_1_uni.items(), key=lambda x: x[1],reverse=True))
bars_1_uni = hv.Bars(data_1_uni[0:50][::-1],"Word","Count").opts(invert_axes=True, width=500, height=800, title="unigram frequency with target=1", color="blue")

(bars_0_uni + bars_1_uni).opts(opts.Bars(tools=['hover']))

### Bigram

In [ ]:
freq_dict_0_bi = defaultdict(int)
for sent in train_0["question_text"]:
    for word in generate_ngrams(sent,n_gram=2):
        freq_dict_0_bi[word] += 1

freq_dict_1_bi = defaultdict(int)
for sent in train_1["question_text"]:
    for word in generate_ngrams(sent,n_gram=2):
        freq_dict_1_bi[word] += 1

In [ ]:
data_0_bi = list(sorted(freq_dict_0_bi.items(), key=lambda x: x[1],reverse=True))
bars_0_bi = hv.Bars(data_0_bi[0:50][::-1],"Word","Count").opts(invert_axes=True, width=500, height=800, title="bigram frequency with target=0", color="red")
data_1_bi = list(sorted(freq_dict_1_bi.items(), key=lambda x: x[1],reverse=True))
bars_1_bi = hv.Bars(data_1_bi[0:50][::-1],"Word","Count").opts(invert_axes=True, width=500, height=800, title="bigram frequency with target=1", color="blue")

(bars_0_bi + bars_1_bi).opts(opts.Bars(tools=['hover']))

### Trigram

In [ ]:
freq_dict_0_tri = defaultdict(int)
for sent in train_0["question_text"]:
    for word in generate_ngrams(sent,n_gram=3):
        freq_dict_0_tri[word] += 1

freq_dict_1_tri = defaultdict(int)
for sent in train_1["question_text"]:
    for word in generate_ngrams(sent,n_gram=3):
        freq_dict_1_tri[word] += 1

In [ ]:
data_0_tri = list(sorted(freq_dict_0_tri.items(), key=lambda x: x[1],reverse=True))
bars_0_tri = hv.Bars(data_0_tri[0:50][::-1],"Word","Count").opts(invert_axes=True, width=500, height=800, title="trigram frequency with target=0", color="red")
data_1_tri = list(sorted(freq_dict_1_tri.items(), key=lambda x: x[1],reverse=True))
bars_1_tri = hv.Bars(data_1_tri[0:50][::-1],"Word","Count").opts(invert_axes=True, width=500, height=800, title="trigram frequency with target=1", color="blue")

(bars_0_tri + bars_1_tri).opts(opts.Bars(tools=['hover']))

# Feature Extraction

## TF-IDF

In [ ]:
def tfidf_features(text, _max_features=10, _max_ngrams=2):
    tfidf = TfidfVectorizer(max_features=_max_features, use_idf=True, ngram_range=(1,_max_ngrams))
    vec = tfidf.fit_transform(text).toarray()
    tfidf_df = pd.DataFrame(vec, columns=["TFIDF_" + n for n in tfidf.get_feature_names()])
    return tfidf_df

In [ ]:
tfidf_train = tfidf_features(train["question_text"])
train = pd.concat([train, tfidf_train], axis=1)
tfidf_test = tfidf_features(test["question_text"])
test = pd.concat([test, tfidf_test], axis=1)

## N-gram Feature

In [ ]:
uni_0_words = [word_freq[0] for word_freq in data_0_uni[0:50]]
uni_1_words = [word_freq[0] for word_freq in data_1_uni[0:50]]
bi_0_words = [word_freq[0] for word_freq in data_0_bi[0:50]]
bi_1_words = [word_freq[0] for word_freq in data_1_bi[0:50]]
tri_0_words = [word_freq[0] for word_freq in data_0_tri[0:50]]
tri_1_words = [word_freq[0] for word_freq in data_1_tri[0:50]]

In [ ]:
for df in [train,test]:
    uni_0_feature = []
    uni_1_feature = []
    bi_0_feature = []
    bi_1_feature = []
    tri_0_feature = []
    tri_1_feature = []
    for line in df["question_text"]:
        uni_0_len = len([word for word in uni_0_words if word in line])
        uni_1_len = len([word for word in uni_1_words if word in line])
        bi_0_len = len([word for word in bi_0_words if word in line])
        bi_1_len = len([word for word in bi_1_words if word in line])
        tri_0_len = len([word for word in tri_0_words if word in line])
        tri_1_len = len([word for word in tri_1_words if word in line])
        
        uni_0_feature.append(uni_0_len)
        uni_1_feature.append(uni_1_len)
        bi_0_feature.append(bi_0_len)
        bi_1_feature.append(bi_1_len)
        tri_0_feature.append(tri_0_len)
        tri_1_feature.append(tri_1_len)
    df["uni_0"] = uni_0_feature
    df["uni_1"] = uni_1_feature
    df["bi_0"] = bi_0_feature
    df["bi_1"] = bi_1_feature
    df["tri_0"] = tri_0_feature
    df["tri_1"] = tri_1_feature
        

In [ ]:
train.head(3)

In [ ]:
test.head(3)

## Meta Feature

In [ ]:
for df in [train,test]:
    df["sent_len"] = df["question_text"].apply(lambda x: len(x.split()))
    df["word_mean_len"] = df["question_text"].apply(lambda x: np.mean([len(i) for i in x.split()]))
    df["punc_num"] = df["question_text"].apply(lambda x: len([c for c in x.split() if c in string.punctuation]))

In [ ]:
train.head(3)

In [ ]:
test.head(3)

# Modeling

# Submission